<a href="https://colab.research.google.com/github/thakerpragnesh/DeepLearning/blob/master/06_VGG16TranferLearningPruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code will help you to load google dive in the jupiter notebook and help to access the dataset stored in the drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
DatasetLoc = '/content/gdrive/MyDrive/Dataset/'
SelDataSet = 'IntelIC'
NumberOfClass = 6
data_dir = DatasetLoc+SelDataSet
logFile = '/content/gdrive/MyDrive/Colab Notebooks/Logs'+'ConvModel.log'
#savePATH
SavePath = '/content/gdrive/MyDrive/Model/'+SelDataSet+'/VGG_Cifar_v2'
LoadPath ='/content/gdrive/MyDrive/Model'+SelDataSet+'VGG_Cifar_v2'
datafile = 'cifar10'
DatasetPath = '/content/gdrive/MyDrive/Dataset/'

Following cell will import basic numeric and ploting librrary

In [ ]:
#import basic library for some basic function
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
import cv2

Following Cell facilitate to read files from drive and and help to read dataset

In [ ]:
# import library to perform file operation
import os #use to access the files 
import tarfile # use to extract dataset from zip files
import sys
import zipfile

Torch Library provides facilities to create networl architechture and write farword and backwor phase od neural network

In [ ]:
#import torch library to build neural network
import torch  # Elementory function of tensor is define in torch package
import torch.nn as nn # Several layer architectur is define here
import torch.nn.functional as F # loss function and activation function

Computer vision is one of the most important application and thus lots od deplopment in the and torch.vision provides many facilities that can be use to imporve model such as data augmentation, reading data batchwise, suffling data before each epoch and many more


In [ ]:
# import torch library related to data processing
import torchvision # provides facilities to access image dataset
from torchvision.datasets.utils import download_url 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision import datasets, models, transforms

#Data Prepration 1

Based on the image size of the dataset choose apropriate values of the color channel and Image Size

Here we can define path to a folder where we can keep all the dataset. In the following we are using the zip files. Originally dataset should be in the following format DataSetName is parent folder and it should contain train and test folder. train and test folder should contain folder for each category and images of respective category should be in the respective category folder

In [ ]:
fullpath = '/content/gdrive/MyDrive/Dataset/IntelIC.zip'
zip_ref = zipfile.ZipFile(fullpath, 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()
print()

Choose an apropriate batch size that can be loaded in the current enviroment without crashing and also do not choose too big batch even if dataset is small because it leads to very few updates per epoch

In [ ]:
bs = 128
ImageSize = 224

Data Augmentaion generally help in reducing overfitting error during trainng process and thus we are performing randon horizontal flip and random crop during training but during validation as no training happens we dont perform data augmentation


In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(ImageSize),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/tmp/'+SelDataSet
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=bs,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Look inside the single batch of dataset images we are going to train

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    #inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

Check if Cuda GPU is available

In [ ]:
#check for CUDA enabled GPU card
def getDeviceType():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')


device = getDeviceType()
print(device)

#Training Process
Below code will work as a base function and provide all the important function like compute loss, accuracy and print result in a perticular formate afte each epoch. Funvtion are as follow
1. Accuracy : Computer accuracy in evalutaion mode of pytorch on given dataset for given model
2. compute_batch_loss : Compute batch loss and append the loss in the list of batch loss.
3. compute_batch_loss_acc : Compute batch loss, batch accuracy and append the loss in the list of batch loss.
4. accumulate_batch_loss_acc: Accumulate loss from the list of batch and acccuraly loss.
5. Epoch end to print the output after every epoch in proper format


In [ ]:
def accuracy(outputs, labels):
# compute accuracy = total correct prediction / total number f ele

    # torch.max(output, dim) will return max value and corresponding index, 
    # Here we are intersted only on index value as it indicate class
    _, preds = torch.max(outputs, dim=1) # get the prediction vector
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# Compute loss of the given batch and return it
def compute_batch_loss(newmodel, batch_X,batch_y):
  images = batch_X.to(device)
  labels = batch_y.to(device)
  out = newmodel(images)                  # Generate predictions
  loss = F.cross_entropy(out, labels) # Calculate loss
  return loss

# Computes loss and accuracy of the given batch(Used in validation)
def compute_batch_loss_acc(newmodel, batch_X,batch_y):
    images = batch_X.to(device)
    labels = batch_y.to(device)
    out = newmodel(images)                    # Generate predictionsin_features=4096
    loss = F.cross_entropy(out, labels)   # Calculate loss
    acc = accuracy(out, labels)           # Calculate accuracy
    return {'val_loss': loss, 'val_acc': acc}

# At the end of epoch accumulate all batch loss and batch accueacy    
def accumulate_batch_loss_acc(outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

def epoch_end(epoch, result):
  # Print in given format 
  # Epoch [0], last_lr: 0.00278, train_loss: 1.2862, val_loss: 1.2110, val_acc: 0.6135
  strResult = "Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
      epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc'])
  print(strResult)
  #print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
  #   epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

#Define Archietecture

In [ ]:
def conv_block(in_channels, out_channels, pool=False, kernalSize =3, padd = 1 ):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernalSize, padding=padd), 
              nn.BatchNorm2d(out_channels), # Layerwise Noramlizarion
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2)) 
    return nn.Sequential(*layers)

def vgg_fc_layer(size_in, size_out):
    layer = nn.Sequential(
        nn.Dropout(p=0.6),
        nn.Linear(size_in, size_out),
        nn.BatchNorm1d(size_out),
        nn.ReLU()
    )
    return layer

In [ ]:
class VGG_Net_16(nn.Module):
    
    def __init__(self, in_channels, num_classes):
        
        super().__init__()
        
        # VGG_BLOCK_1(03,64)    -> Input(3,64x64) -> Output(64,32x32)
        self.conv11 = conv_block(in_channels, 64,pool=False)
        self.conv12 = conv_block(64, 64,pool=True) #112x112

        # VGG_BLOCK_2(64,128)   -> Input(64,32x32) -> Output(128,16x16)
        self.conv21 = conv_block(64,128,pool=False)
        self.conv22 = conv_block(128, 128, pool=True) #56x56                             
        
        # VGG_BLOCK_3(128,128) -> Input(128,16x16) -> Output(128,16x16)
        self.conv31 = conv_block(128,256,pool=False)
        self.conv32 = conv_block(256, 256, pool=False)  #28x28
        self.conv33 = conv_block(256, 256, pool=True)
        
        # VGG_BLOCK_4(256,512)  -> Input(256,8x8) -> Output(512x4x4)
        self.conv41 = conv_block(256, 512, pool=False)                              
        self.conv42 = conv_block(512, 512, pool=False)
        self.conv43 = conv_block(512, 512, pool=True)  #14x14

        # VGG_BLOCK_5(512,512) -> Input(512x4x4) -> Output(512x4x4)
        self.conv51 = conv_block(512, 512, pool=False)                              
        self.conv52 = conv_block(512, 512, pool=False)
        self.conv53 = conv_block(512, 512, pool=True)  #7x7

          
        # Classifier Part Of VGG_Net
        self.FC1 = vgg_fc_layer(7*7*512, 4096)
        self.FC2 = vgg_fc_layer(4096, 4096)
        self.output = vgg_fc_layer(7*7*512, 4096)
    
    def forward(self, xb):
        out = self.conv11(xb)
        out = self.conv12(out)
        
        out = self.conv21(out)
        out = self.conv22(out)

        out = self.conv31(out)
        out = self.conv32(out)
        out = self.conv33(out)

        out = self.conv41(out)
        out = self.conv42(out)
        out = self.conv43(out)

        out = self.conv51(out)
        out = self.conv52(out)
        features = self.conv53(out)

        out = features.view(out.size(0), -1)  # Flatten the layer
        return out

In [ ]:
#NumberOfClass = 6
MyNewModel = VGG_Net_16(3,NumberOfClass)
print(MyNewModel)

VGG_Net_16(
  (conv11): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv12): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv21): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv22): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kerne

In [ ]:
newModel = torchvision.models.vgg16(pretrained=True)
print(newModel)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
#[0,2, 5,7, 10,12,14, 17,19,21, 24,26,28]
MyNewModel.conv11[0].weight = newModel.features[0]._parameters['weight']
MyNewModel.conv12[0].weight = newModel.features[2]._parameters['weight']

MyNewModel.conv21[0].weight = newModel.features[5]._parameters['weight']
MyNewModel.conv22[0].weight = newModel.features[7]._parameters['weight']

MyNewModel.conv31[0].weight = newModel.features[10]._parameters['weight']
MyNewModel.conv32[0].weight = newModel.features[12]._parameters['weight']
MyNewModel.conv33[0].weight = newModel.features[14]._parameters['weight']

MyNewModel.conv41[0].weight = newModel.features[17]._parameters['weight']
MyNewModel.conv42[0].weight = newModel.features[19]._parameters['weight']
MyNewModel.conv43[0].weight = newModel.features[21]._parameters['weight']

MyNewModel.conv51[0].weight = newModel.features[24]._parameters['weight']
MyNewModel.conv52[0].weight = newModel.features[26]._parameters['weight']
MyNewModel.conv53[0].weight = newModel.features[28]._parameters['weight']

## Define Training 
Here we will evalute our model after each epoch on validation dataset using evalute method
get_lr method returnd last learning rate used in the training
Here we are using one fit cycle method in which we specify the max learning rate and learning rate start from 1/10th value of max_lr and slowly increases the value to max_lr for 40% of updates then decreases to its initial value for 40% updates and then further decreases to 1/100th of max_lr value to perform final fine tuning.

In [ ]:
@torch.no_grad()
# evalute model on given dataset using given data loader
def evaluate(model, data_loader):
    model.eval()
    with torch.no_grad():
      for batch_X, batch_y in data_loader:
        outputs = [compute_batch_loss_acc(model,batch_X,batch_y)]
      return accumulate_batch_loss_acc(outputs)

# Use special scheduler to change the value of learning rate
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

# epoch=8, max_lr=.01, weight_decay(L2-Regu parametr)=.0001,opt_func=Adam


def fit_one_cycle(ModelName,epochs, max_lr, model, 
                  weight_decay=0, L1=0,grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer here we will use one cycle scheduler with max learning
    # rate given by max_lr, default optimizer is SGD but we will use ADAM, and 
    # L2 Regularization using weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(dataloaders['train']))
    
    print("Training Starts")
    with open("/content/gdrive/My Drive/data/ConvModel.log","a") as f:
      for epoch in range(epochs):
          # Training Phase 
          model.train()  #######################
          train_losses = []
          lrs = []
          #for batch in train_loader:
          for batch_X, batch_y in dataloaders['train']:
              
              # computer the training loss of current batch
              loss = compute_batch_loss(model,batch_X,batch_y)
              
              l1_crit = nn.L1Loss()
              reg_loss = 0
              for param in model.parameters():
                reg_loss += l1_crit(param,target=torch.zeros_like(param))

              loss += L1*reg_loss 
              train_losses.append(loss)
              loss.backward() # compute the gradient of all weights
              
              # Clip the gradient value to maximum allowed grad_clip value
              if grad_clip: 
                  nn.utils.clip_grad_value_(model.parameters(), grad_clip)
              
              optimizer.step() # Updates weights 
              # pytorch by default accumulate grade history and if we dont want it
              # we should make all previous grade value equals to zero
              optimizer.zero_grad() 
              
              # Record & update learning rate
              lrs.append(get_lr(optimizer))
              sched.step() # Update the learning rate
          
          # Compute Validation Loss and Valodation Accuracy
          result = evaluate(model, dataloaders['test'])

          # Compute Train Loss of whole epoch i.e mean of loss of batch 
          result['train_loss'] = torch.stack(train_losses).mean().item()

          # Observe how learning rate is change by schedular
          result['lrs'] = lrs

          # print the observation of each epoch in a proper format
          epoch_end(epoch, result) 
          strResult = "Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc'])
          f.write(f"{ModelName}-\t{strResult}\n")
          history.append(result) # append tupple result with val_acc, vall_loss, and trin_loss
        
    return history

Define the function that help is evaluating the model such as 
1. acuracy vs eopch to see that whether performance is improving or not during training, 
2. model loss vs epoch to see the loos value improvment during each epoch and 
3. learning rate plot to see how learning rate changes during each updates

In [ ]:
# Plot Validation accuracy Over Epoch
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

# Plot training and validation loss to check for overfitting and underfitting
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

# plot the learning rate values over each batch
def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.');

we can download pretrain model and perform fine tuning on the model with fix feature extracter using the following

In [ ]:
#################### Download Pretrain VGGNet ################
'''
newModel = torchvision.models.vgg16(pretrained=True)
for param in newModel.parameters():
    param.requires_grad = False
#print(newModel)
#Need to change the below code if we choose different model
print(newModel.classifier[6])
num_ftrs = newModel.classifier[6].in_features

# Here the size of each output sample is set to 10.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
newModel.classifier[6] = nn.Linear(num_ftrs, NumberOfClass)
newModel = newModel.to(device)
'''
##################### Download Pretrain ResNet 18 ############################

'''
newModel = torchvision.models.resnet18(pretrained=True)
print(newModel.fc)
for param in newModel.parameters():
    param.requires_grad = False
#print(newModel)
#Need to change the below code if we choose different model
num_ftrs = newModel.fc.in_features
# Here the size of each output sample is set to 10.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
newModel.fc = nn.Linear(num_ftrs, 10)
newModel = newModel.to(device)
'''
################# Load Stored Trained Model #####################################

PATH ='/content/gdrive/MyDrive/Model/IntelIC/VGG_Cifar_v2'
if device== torch.device('cpu'):
    newModel = torch.load(PATH, map_location=torch.device('cpu'))
else:
    newModel = torch.load(PATH, map_location=torch.device('cuda'))

# Need to change the following loop for dofferent model
count=0
for param in newModel.parameters():
    print(count," : ",param.shape)
    if count in (16,18,20):
      param.requires_grad=True
      #print(param.requires_grad)
    else:
      param.requires_grad=False
      #print(param.requires_grad)
    count +=1

print(newModel)
#################################################################################


In [ ]:
count = 0
for param_tensor in newModel.state_dict():
    print(count,":\t",param_tensor, "\t", newModel.state_dict()[param_tensor].size())
    count+=1
print()


During the next itteration if we had a better model stored in drive we can load model and continue training by learning parameter of all dense layer

Initialise all the hyper meter required for training such as learning rate that control the step size, gradient clipping value that limits the maximum value of grads, L2 regularization using weight decay value, L1 regularization using L1 value and then choose apropriate optimization funtion for training.

In [ ]:
historyfull = []

###In the below cell we are setting following hyper-parameter
1. Epochs = Number of epoch we want to perform on dataset
2. Max_Lr = indicate maximum learning rate
3. grad clip = to control the maximum value of gradient
4. weight decay = to do L2 regularization on model
5. L1 regularization constant

In [ ]:
historylast = []
epochs = 15
# We are using one cycle fit function in which learning rate start with 1/10th 
# of selected maximum learning rate and increase learning rate from min to max
# in 1st phase and then decrease from max to min in 2nd phase
max_lr = 3e-4
grad_clip = 0.3 # Maximum allowed value of grad: grad = min(grad,0.01)
# weight decay is a constant multiplier for L2 regularization
weight_decay = 1e-6
L1 = 1e-6
# Choosing Adma optimizer as try to implement SGD+Momentum+Adaptive Learining
opt_func = torch.optim.Adam

MODEL_NAME = f"VGG_Net-MLR-\t{max_lr}-GC{grad_clip}-WD-{weight_decay}-L1-{L1}"

Start the training using one fit cycle algorithms where learning rate start with one tenth of provided max_lr and then increase it value till that point and then decrease onword and for last few epoch learning rate furthe decrease

Pass the hyperparameter for training of the model and start training process for set number of epcoh

In [ ]:
%%time
historylast += fit_one_cycle(MODEL_NAME,epochs, max_lr, newModel,  
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, L1=L1,
                             opt_func=opt_func
                             )

In [ ]:
for item in historylast:
  historyfull.append(item)

Evalute model for accuracy for last training run 

In [ ]:
print("Accuracy in last run")
plot_accuracies(historylast)

Evalute Models for whole training process for accuracy


In [ ]:
print("Accuracy is all run")
plot_accuracies(historyfull)

Evalute last run for loss

In [ ]:
print("Loss of last run")
plot_losses(historylast)

Evalute full training process for loss value

In [ ]:
print("Loss in All run")
plot_losses(historyfull)

See how the learning rate changedin process

In [ ]:
print("Learning rate of all run")
plot_lrs(historyfull)

Print validation accuracy for all epoch of last run

In [ ]:
accuracies = [x['val_acc']*100 for x in historylast]
accuracies = ["%.2f" % v for v in accuracies]
print(accuracies)

##Define method to compute accuracy for a given model on given dataset

In [ ]:
print(SavePath)

In [ ]:
def accuraciesTotal(newModel, data_loader):
  with torch.no_grad():
    acc = []
    for batch in data_loader:
        images, label = batch
        images, labels = batch[0].to(device), batch[1].to(device)
        out = newModel(images)
        acc.append(accuracy(out, labels))
        
    return torch.mean(torch.stack(acc))

###Prepare the data loader for inference. During the inference we want to perform same tranformation on test and train dataset

In [ ]:
image_datasets_eval = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms['test'])
                  for x in ['train', 'test']}

dataloaders_eval = {x: torch.utils.data.DataLoader(image_datasets_eval[x], batch_size=bs,
                                             shuffle=False, num_workers=2)
                  for x in ['train', 'test']}

###Evalute final accuracy of test and train dataset

In [ ]:
trainacc = accuraciesTotal(newModel,dataloaders_eval['train'])
testacc = accuraciesTotal(newModel,dataloaders_eval['test'])
print("Train Acc :" + str(trainacc) + "\nTest  Acc :" +str(testacc))
#print(testacc)

Save the model if we feel we have made some improvement in the model


In [ ]:
#PATH = "/content/gdrive/My Drive/Model/cifar10/VGG_Cifar_P3"
torch.save(newModel, SavePath)

In [ ]:
print(logFile)

In [ ]:
#outfile = LogLoc+"FinalOut.log"

with open(logFile,"a") as f2:
  f2.write("Train Acc :" + str(trainacc) + "\nTest  Acc :" +str(testacc))

# Below code shows basic ln structure pruning methods provided by the pytorch. Here if we want to apply custom pruning then we need to write our own compute_mask function 

In [ ]:
#import library to facilitate pruning
import torch.nn.utils.prune as prune
import time

###Select the features want to prune and store them in the list of of Mudule here

In [ ]:
print(newModel.features)

In [ ]:
Module = []
for i in (0,3,6,8,11,13,16,18):
  Module.append(newModel.features[i])

for i in range(len(Module)):
  print(Module[i])

Select the amount of feature we want to prune in each Layer

In [ ]:
pr = [.05,.05,.1,.1,.15,.15,.15,.20]
for i in range(len(Module)):
  prune.ln_structured(Module[i], name="weight", amount=pr[i], n=1, dim=0)

In [ ]:
numberOfZero = 0
numberOfElements = 0
for i in range(len(Module)):
    numberOfZero += torch.sum(Module[i].weight == 0)
    numberOfElements += Module[i].weight.nelement()
    print(
        "Sparsity in conv1.weight: {:.2f}%".format(
            100. * float(torch.sum(Module[i].weight == 0))
            / float(Module[i].weight.nelement())
        )
    )    
       
print("Global Sparsity: {:.2f}%".format(100*float(numberOfZero)/float(numberOfElements)))

In [ ]:
'''
prune.remove(model.conv11[0], 'weight')
prune.remove(model.conv12[0], 'weight')

prune.remove(model.conv22[0], 'weight')
prune.remove(model.conv21[0], 'weight')

prune.remove(model.conv31[0], 'weight')
prune.remove(model.conv32[0], 'weight')
prune.remove(model.conv33[0], 'weight')

prune.remove(model.conv41[0], 'weight')
prune.remove(model.conv42[0], 'weight')
prune.remove(model.conv43[0], 'weight')

prune.remove(model.conv51[0], 'weight')
prune.remove(model.conv52[0], 'weight')
prune.remove(model.conv53[0], 'weight')
'''
for i in (0,3,6,8,11,13,16,18):
  prune.remove(newModel.features[i], 'weight')